In [1]:
import os
import findspark
findspark.init()
findspark.find()
from pyspark.sql.functions import lit
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import monotonically_increasing_id

# Task I

In [2]:
# connect spark to PostgreSQL
appName = "Big Data Analytics"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = sqlContext.sparkSession.builder.getOrCreate()

C:\Spark\python\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# fifa=['sofifa_id int', 'player_url string', 'short_name string', 'long_name string', 'player_positions string',
#  'overall int', 'potential int', 'value_eur double', 'wage_eur double', 'age int',
#  'dob date', 'height_cm int', 'weight_kg int', 'club_team_id int', 'club_name string',
#  'league_name string', 'league_level int', 'club_position string', 'club_jersey_number int', 'club_loaned_from string',
#  'club_joined date', 'club_contract_valid_until int', 'nationality_id int', 'nationality_name string', 'nation_team_id int',
#  'nation_position string', 'nation_jersey_number int', 'preferred_foot string', 'weak_foot int', 'skill_moves int',
#  'international_reputation int', 'work_rate string', 'body_type string', 'real_face string', 'release_clause_eur double',
#  'player_tags string', 'player_traits string', 'pace int', 'shooting int', 'passing int',
#  'dribbling int', 'defending int', 'physic int', 'attacking_crossing int', 'attacking_finishing int',
#  'attacking_heading_accuracy int', 'attacking_short_passing int', 'attacking_volleys int', 'skill_dribbling int', 'skill_curve int',
#  'skill_fk_accuracy int', 'skill_long_passing int', 'skill_ball_control int', 'movement_acceleration int', 'movement_sprint_speed int',
#  'movement_agility int', 'movement_reactions int', 'movement_balance int', 'power_shot_power int', 'power_jumping int',
#  'power_stamina int', 'power_strength int', 'power_long_shots int', 'mentality_aggression int', 'mentality_interceptions int',
#  'mentality_positioning int', 'mentality_vision int', 'mentality_penalties int', 'mentality_composure int', 'defending_marking_awareness int',
#  'defending_standing_tackle int', 'defending_sliding_tackle int', 'goalkeeping_diving int', 'goalkeeping_handling int', 'goalkeeping_kicking int',
#  'goalkeeping_positioning int', 'goalkeeping_reflexes int', 'goalkeeping_speed int', 'ls string', 'st string',
#  'rs string', 'lw string', 'lf string', 'cf string', 'rf string',
#  'rw string', 'lam string', 'cam string', 'ram string', 'lm string',
#  'lcm string', 'cm string', 'rcm string', 'rm string', 'lwb string',
#  'ldm string', 'cdm string', 'rdm string', 'rwb string', 'lb string',
#  'lcb string', 'cb string', 'rcb string', 'rb string', 'gk string',
#  'player_face_url string', 'club_logo_url string', 'club_flag_url string', 'nation_logo_url string', 'nation_flag_url string']

In [4]:
fifa='sofifa_id int,player_url string,short_name string,long_name string,player_positions string,overall int,potential int,value_eur double,wage_eur double,age int,dob date,height_cm int,weight_kg int,club_team_id int,club_name string,league_name string,league_level int,club_position string,club_jersey_number int,club_loaned_from string,club_joined date,club_contract_valid_until int,nationality_id int,nationality_name string,nation_team_id int,nation_position string,nation_jersey_number int,preferred_foot string,weak_foot int,skill_moves int,international_reputation int,work_rate string,body_type string,real_face string,release_clause_eur double,player_tags string,player_traits string,pace int,shooting int,passing int,dribbling int,defending int,physic int,attacking_crossing int,attacking_finishing int,attacking_heading_accuracy int,attacking_short_passing int,attacking_volleys int,skill_dribbling int,skill_curve int,skill_fk_accuracy int,skill_long_passing int,skill_ball_control int,movement_acceleration int,movement_sprint_speed int,movement_agility int,movement_reactions int,movement_balance int,power_shot_power int,power_jumping int,power_stamina int,power_strength int,power_long_shots int,mentality_aggression int,mentality_interceptions int,mentality_positioning int,mentality_vision int,mentality_penalties int,mentality_composure int,defending_marking_awareness int,defending_standing_tackle int,defending_sliding_tackle int,goalkeeping_diving int,goalkeeping_handling int,goalkeeping_kicking int,goalkeeping_positioning int,goalkeeping_reflexes int,goalkeeping_speed int,ls string,st string,rs string,lw string,lf string,cf string,rf string,rw string,lam string,cam string,ram string,lm string,lcm string,cm string,rcm string,rm string,lwb string,ldm string,cdm string,rdm string,rwb string,lb string,lcb string,cb string,rcb string,rb string,gk string,player_face_url string,club_logo_url string,club_flag_url string,nation_logo_url string,nation_flag_url string'

In [5]:
df = spark.read.csv("./data/players_15.csv",header=True,schema=fifa)

In [6]:
df.show(1,vertical=True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | null                 
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [7]:
# import datetime
# schema = "dob date"
# data = [ [datetime.datetime.strptime('2021-01-01', "%Y-%m-%d").date()],
#         [datetime.datetime.strptime('2000-01-01', "%Y-%m-%d").date()]]

# test_df = spark.createDataFrame(data, schema)

In [8]:
df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: integer (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: date (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)


In [9]:
df=df.withColumn("year",lit(2015))

In [10]:
df.show(vertical=True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | null                 
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [11]:
df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: integer (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: date (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)


In [12]:
#don't consider players_15,because we will use append()
players_list=os.listdir("./data")[10:]
players_list

['players_16.csv',
 'players_17.csv',
 'players_18.csv',
 'players_19.csv',
 'players_20.csv',
 'players_21.csv',
 'players_22.csv']

In [13]:
print(os.path.join("./data/",players_list[0]).split('_')[1][:2])
print(2000+int(os.path.join("./data/",players_list[0]).split('_')[1][:2]))

16
2016


In [14]:
for i in players_list:
    path=os.path.join("./data/",i)
    temp_df=spark.read.csv(path,header=True,schema=fifa)
    year=2000+int(path.split('_')[1][:2])
    temp_df=temp_df.withColumn("year",lit(year))
    df = df.union(temp_df)

In [15]:
# df.describe().show(vertical=True)

In [16]:
df.count()

142079

In [17]:
df.select("year").distinct().show()

+----+
|year|
+----+
|2015|
|2016|
|2017|
|2018|
|2019|
|2020|
|2021|
|2022|
+----+



In [18]:
#142079 !=45629,sofifa_id can't be used as unique id
df.select("sofifa_id").distinct().count()

45629

In [19]:
# df.write.mode("overwrite").saveAsTable("fifa")
# spark.catalog.listTables()

In [20]:
df=df.withColumn("id",monotonically_increasing_id())

In [21]:
df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: integer (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: date (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)


In [22]:
# #cast club_team_id,nation_team_id into "integer"
# df=df.withColumn("new_club_team_id",df["club_team_id"].cast("integer")).drop("club_team_id").withColumnRenamed("new_club_team_id","club_team_id")
# df=df.withColumn("new_nation_team_id",df["nation_team_id"].cast("integer")).drop("nation_team_id").withColumnRenamed("new_nation_team_id","nation_team_id")

In [23]:
db_properties={}
db_properties['username']="postgres"
db_properties['password']="zx200042"
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
db_properties['table']="fifa"
db_properties['driver']="org.postgresql.Driver"

df.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

# Task II

In [24]:
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df_read.show(1, vertical=True)
df_read.printSchema()

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | null                 
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [25]:
df_read.createOrReplaceTempView("df_view")

In [26]:
# sqlWay = spark.sql("""
# SELECT club_name,count(*)
# FROM df_view
# Where year = 2022 and club_contract_valid_until=2023
# group by club_name  
# order by count(*) desc
# """)

# sqlWay.show(4)
def contracts_ending_2023(x):
    sqlWay = spark.sql("""
    SELECT club_name,count(*) as players_with_contracts_ending_in_2023
    FROM df_view
    Where year = 2022 and club_contract_valid_until=2023
    group by club_name  
    order by count(*) desc
    """)
    sqlWay.show(x)

In [27]:
contracts_ending_2023(5)

+--------------------+-------------------------------------+
|           club_name|players_with_contracts_ending_in_2023|
+--------------------+-------------------------------------+
|En Avant de Guingamp|                                   19|
| Club Atlético Lanús|                                   17|
|       Lechia Gdańsk|                                   17|
|            Barnsley|                                   16|
|        Kasimpaşa SK|                                   16|
+--------------------+-------------------------------------+
only showing top 5 rows



In [28]:
def number_of_players_over_27(x):
    sqlWay = spark.sql("""
    select club_name,avg(number_players) as average_number_players_allyear from
    (
    SELECT club_name,count(*) as number_players
    FROM df_view
    Where age>27 and club_name is not null
    group by club_name,year
    ) as temp
    group by club_name
    order by average_number_players_allyear desc
    """)
    #find exactly how many rows(num) should be shown
    sqlWay=sqlWay.withColumn("id",monotonically_increasing_id()+1)
    least=sqlWay.select('average_number_players_allyear').take(x)[x-1]['average_number_players_allyear']
    rows_to_show=x
    temp=sqlWay.filter(sqlWay.id==rows_to_show+1).take(1)[0]['average_number_players_allyear']
    while temp==least:
        rows_to_show=rows_to_show+1
        temp=sqlWay.filter(sqlWay.id==rows_to_show+1).take(1)[0]['average_number_players_allyear']
    sqlWay.show(rows_to_show)

In [29]:
number_of_players_over_27(12)

+--------------------+------------------------------+---+
|           club_name|average_number_players_allyear| id|
+--------------------+------------------------------+---+
|  Dorados de Sinaloa|                          19.0|  1|
| Matsumoto Yamaga FC|                          19.0|  2|
| Shanghai Shenhua FC|                          18.5|  3|
|          Qingdao FC|                          18.0|  4|
|Club Deportivo Jo...|                          17.5|  5|
|            Altay SK|                          17.0|  6|
|         Guaireña FC|                          17.0|  7|
|İstanbul Başakşeh...|                        16.625|  8|
|      BB Erzurumspor|                          16.5|  9|
|        Club Olimpia|                          16.5| 10|
|      Sport Huancayo|                          16.5| 11|
|    Beijing Renhe FC|                          16.0| 12|
|              Paraná|                          16.0| 13|
|  Sandecja Nowy Sącz|                          16.0| 14|
+-------------

In [30]:
# sqlWay1=spark.sql('''
# select nation_position,year,count(*) as cnt 
# from df_view
# where nation_position is not null and nation_position<>'SUB'
# group by nation_position,year
# order by cnt desc
# ''')
# sqlWay1.show()

In [31]:
# sqlWay1.createOrReplaceTempView("sqlWay1")
# spark.catalog.listTables()

In [32]:
# sqlWay=spark.sql('''
# select temp.year,nation_position,sqlWay1.cnt from
# (
# select year,max(cnt) as maxcnt
# from sqlWay1
# group by year
# ) as temp
# left join sqlWay1
# on temp.maxcnt=sqlWay1.cnt and temp.year=sqlWay1.year
# order by temp.year
# ''')

In [33]:
# sqlWay.show(30)

In [34]:
def most_frequent_nation_position():
    sqlWay1=spark.sql('''
    select nation_position,year,count(*) as cnt 
    from df_view
    where nation_position is not null and nation_position<>'SUB'
    group by nation_position,year
    order by cnt desc
    ''')
    sqlWay1.createOrReplaceTempView("sqlWay1")
    sqlWay2=spark.sql('''
    select temp.year,nation_position,sqlWay1.cnt from
    (
    select year,max(cnt) as maxcnt
    from sqlWay1
    group by year
    ) as temp
    left join sqlWay1
    on temp.maxcnt=sqlWay1.cnt and temp.year=sqlWay1.year
    order by temp.year
    ''')
    sqlWay2.show(30)

In [35]:
most_frequent_nation_position()

+----+---------------+---+
|year|nation_position|cnt|
+----+---------------+---+
|2015|             GK| 47|
|2015|            RCB| 47|
|2015|            LCB| 47|
|2016|            RCB| 46|
|2016|             GK| 46|
|2017|             GK| 47|
|2017|            LCB| 47|
|2017|            RCB| 47|
|2018|             GK| 50|
|2018|            LCB| 50|
|2018|            RCB| 50|
|2019|            LCB| 48|
|2019|             GK| 48|
|2019|            RCB| 48|
|2020|            RCB| 49|
|2020|            LCB| 49|
|2020|             GK| 49|
|2021|            RCB| 49|
|2021|             GK| 49|
|2021|            LCB| 49|
|2022|            LCB| 33|
|2022|             GK| 33|
|2022|            RCB| 33|
+----+---------------+---+

